In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import yfinance as yf
import talib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.trial import TrialState
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau 
import shap
import plotly.graph_objs as go
import plotly.offline as pyo
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F
import math


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())

In [ ]:
start_date = '2018-01-01'
end_date = '2024-01-24'

stock_data = yf.download("AAPL", start=start_date, end=end_date)[["Adj Close", "High", "Low", "Volume"]]

stock_data = stock_data.reset_index()

stock_data = stock_data[['Date', 'Adj Close', "High", "Low", "Volume"]]

stock_data = stock_data.sort_values(by="Date")
stock_data.head(45)

In [ ]:
time_step = 66

In [ ]:
test_index = int((len(stock_data)-time_step)*0.8+time_step+time_step)

In [ ]:
date = stock_data["Date"].iloc[time_step:].dt.strftime('%Y-%m-%d')
date_test = stock_data["Date"].iloc[test_index:].reset_index()
date_test.drop(columns=["index"], inplace=True)
date_test

In [ ]:
def add_technical_indicators(data, timeperiod=time_step):

    # MACD
    macd, macdsignal, macdhist = talib.MACD(data["Adj Close"], fastperiod=12, slowperiod=26, signalperiod=9)

    rsi = talib.RSI(data["Adj Close"], timeperiod=14)

    # CMO
    cmo = talib.CMO(data["Adj Close"], timeperiod=timeperiod)

    # MOM
    mom = talib.MOM(data["Adj Close"], timeperiod=timeperiod)

    # Bollinger Bands
    upperband, middleband, lowerband = talib.BBANDS(data["Adj Close"], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

    # SMA
    sma_s = talib.SMA(data["Adj Close"], timeperiod=20)
    sma_l = talib.SMA(data["Adj Close"], timeperiod=50)

    # Calculate Exponential Moving Average (EMA)
    ema = talib.EMA(data["Adj Close"], timeperiod=timeperiod)

    # Calculate Stochastic Oscillator
    slowk, slowd = talib.STOCH(data['High'], data['Low'], data['Adj Close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

    # Calculate Average True Range (ATR)
    atr = talib.ATR(data['High'], data['Low'], data['Adj Close'], timeperiod=14)

    # Calculate On-Balance Volume (OBV)
    obv = talib.OBV(data['Adj Close'], data['Volume'])

    # Combine all indicators into a DataFrame
    indicators = pd.DataFrame({
        'MACD': macd,
        'MACD_signal': macdsignal,
        'RSI': rsi,
        'CMO': cmo,
        'MOM': mom,
        'Upper_BB': upperband,
        'Middle_BB': middleband,
        'Lower_BB': lowerband,
        'SMA_SHORT': sma_s,
        'SMA_LONG': sma_l,
        'EMA': ema,
        'SLOWK': slowk,
        'SLOWD': slowd,
        'ATR': atr,
        'OBV': obv,

    })
    return indicators

In [ ]:
indicators = add_technical_indicators(stock_data)
indicators.head(45)

In [ ]:
indicators_with_price = pd.concat([indicators, stock_data["Adj Close"]], axis=1, join='inner')
indicators_with_price.head(45)

In [ ]:
indicators_with_price = indicators_with_price.dropna()
indicators_with_price = indicators_with_price.reset_index(drop=True)
indicators_with_price

In [ ]:
# Irrelelvant
indicators_with_price['Prev_Adj_Close'] = indicators_with_price['Adj Close'].shift(1)
indicators_with_price['Return'] = ((indicators_with_price['Adj Close'] - indicators_with_price['Prev_Adj_Close'])/indicators_with_price['Prev_Adj_Close'])*100
indicators_with_price['Signal'] = np.where(indicators_with_price['Return'] > 1, 1,
                                           np.where(indicators_with_price['Return'] < -1, 2, 0))
indicators_with_price


In [ ]:
# Not important
indicators_with_price["Signal"].value_counts()

In [ ]:
indicators_with_price.dropna(inplace=True)
indicators_with_price

In [ ]:
indicators_with_price.columns

In [ ]:
# indicators_with_price = indicators_with_price.drop(columns=['Next_Adj_Close', 'Return'])
# indicators_with_price

In [ ]:
indicators_with_price.drop(columns=['Prev_Adj_Close', "Signal"], inplace=True)
indicators_with_price.head(50)

In [ ]:
y = indicators_with_price["Return"]
y_2 = indicators_with_price["SMA_SHORT"]
y_3 = indicators_with_price["EMA"]
y_4 = indicators_with_price["Upper_BB"]
y_5 = indicators_with_price["Middle_BB"]
y_6 = indicators_with_price["Lower_BB"]
X = np.array(date)

trace = go.Scatter(x=X, y=y, mode="lines", name="Adj Close")
trace_2 = go.Scatter(x=X, y=y_2, mode="lines", name="SMA")
trace_3 = go.Scatter(x=X, y=y_3, mode="lines", name="EMA")
trace_4 = go.Scatter(x=X, y=y_4, mode="lines", name="Upper_BB")
trace_5 = go.Scatter(x=X, y=y_5, mode="lines", name="Middle_BB")
trace_6 = go.Scatter(x=X, y=y_6, mode="lines", name="Lower_BB")



layout = go.Layout(
    title='Stock Price and Volume',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Adj Close', side='left', rangemode='tozero'),
    yaxis2=dict(title='SMA', side='right', overlaying='y', rangemode='tozero'),
    yaxis3=dict(title='EMA', side='right', overlaying='y', rangemode='tozero'),
    yaxis4=dict(title='Upper_BB', side='right', overlaying='y', rangemode='tozero'),
    yaxis5=dict(title='Middle_BB', side='right', overlaying='y', rangemode='tozero'),
    yaxis6=dict(title='Lower_BB', side='right', overlaying='y', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=[trace, trace_2, trace_3, trace_4, trace_5, trace_6], layout=layout)

# Show plot
pyo.iplot(fig)

In [ ]:
# Custom Dataset
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, window_size, device="gpu"):
        self.X = X.clone().detach().to(torch.float).to(device)
        self.y = y.clone().detach().to(torch.float).to(device)
        self.window_size = window_size

    def __len__(self):
        # Adjust the length to account for window size
        return len(self.X) - self.window_size 

    def __getitem__(self, idx):
        # Ensure idx is within the valid range
        if idx + self.window_size > len(self.X):
            raise IndexError("Index out of bounds")

        X_window = self.X[idx : idx + self.window_size]
        
        y_target = self.y[idx + self.window_size]  

        return X_window.clone().detach().to(torch.float), y_target.clone().detach().to(torch.float)


In [ ]:
X = indicators_with_price.iloc[:,:-1]
y = indicators_with_price.iloc[:,-2]

signal_true = indicators_with_price.iloc[:,-1]
y

In [ ]:
train_signal_true = signal_true.iloc[:int(len(X)*0.8)]
test_signal_true = signal_true.iloc[int(len(X)*0.8):]
test_signal_true

In [ ]:
correlation_matrix = X.corr()

# Perform hierarchical clustering to find the order of features
linked = sch.linkage(sch.distance.pdist(correlation_matrix), method='ward')
cluster_order = sch.dendrogram(linked, no_plot=True)['leaves']

# Reorder the correlation matrix
correlation_matrix_ordered = correlation_matrix.iloc[cluster_order, cluster_order]

fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix_ordered,
                    x=correlation_matrix_ordered.columns,
                    y=correlation_matrix_ordered.columns,
                    colorscale='Viridis',
                    text=correlation_matrix_ordered.round(2).values,  
                    texttemplate="%{text}",
                    textfont={"size":9}  
                    ))

# Update the layout
fig.update_layout(
    title='Ordered Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=1000,  
    height=1000,  
)

# Show the figure
pyo.iplot(fig)

In [ ]:
X= X.iloc[:, cluster_order]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

y_test.head(44)

In [ ]:
X_train_df = pd.DataFrame()
X_test_df = pd.DataFrame()
scaler_dict = {}

X_train_df = X_train
X_test_df = X_test

for column in X_train.columns:

    if column not in ["Adj Close", "Return"]:
        scaler = MinMaxScaler()

        X_train_scaled = scaler.fit_transform(X_train[[column]].values)
        X_train_df[column] = X_train_scaled
            
        X_test_scaled = scaler.transform(X_test[[column]].values)
        X_test_df[column] = X_test_scaled

        scaler_dict[column] = scaler


X_train_df.head(24)

features = X_train_df.columns
X_train_df

In [ ]:
scaler_adj = MinMaxScaler()
scaler_adj.fit(X_train[["Adj Close"]].values)

X_train_df['Adj Close'] = scaler_adj.transform(X_train[['Adj Close']].values).flatten()
X_test_df['Adj Close'] = scaler_adj.transform(X_test[['Adj Close']].values).flatten()

y_train = scaler_adj.transform(y_train.values.reshape(-1,1)).flatten()
y_test = scaler_adj.transform(y_test.values.reshape(-1,1)).flatten()



len(y_test)

In [ ]:
correlation_matrix = X_train_df.corr()

# Create the heatmap with text
fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='Viridis',
                    text=correlation_matrix.round(2).values,  # Rounded values for display
                    texttemplate="%{text}",
                    textfont={"size":9}  # Adjust text size if necessary
                    ))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=1000,  # or any width you desire
    height=1000,  # or any height you desire
)

# Show the figure
pyo.iplot(fig)

In [ ]:
X_train_tensor = torch.tensor(X_train_df.to_numpy(), dtype=torch.float, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float, device=device)

X_test_tensor = torch.tensor(X_test_df.to_numpy(), dtype=torch.float, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float, device=device)

train_data = RollingWindowDataset(X_train_tensor, y_train_tensor, window_size=time_step, device=device)
test_data = RollingWindowDataset(X_test_tensor, y_test_tensor, window_size=time_step, device=device)

print(test_data.__getitem__(0)[1])
print(test_data.__getitem__(1)[0])


In [ ]:


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, 1, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        # Originally: pe = pe.unsqueeze(0).transpose(0, 1)
        # Corrected to keep as [max_len, d_model] for easier indexing
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        # Correctly index into pe to accommodate x's sequence length
        # Ensure x and pe have compatible sequence lengths for broadcasting
        x = x + self.pe[:x.size(0)]
        return x


In [ ]:
class Time2Vec(nn.Module):
    def __init__(self, d_model):
        super(Time2Vec, self).__init__()
        self.d_model = d_model
        self.linear = nn.Linear(1, 1)  # Linear part
        self.periodic = nn.Linear(1, d_model - 1)  # Periodic part

    def forward(self, x):
        # x is expected to be of shape [batch_size, seq_len, 1] (1 for time dimension)
        linear_part = self.linear(x)  # [batch_size, seq_len, 1]
        
        # Apply periodic transformation
        sin_trans = torch.sin(self.periodic(x))
        cos_trans = torch.cos(self.periodic(x))
        periodic_part = torch.cat((sin_trans, cos_trans), -1)[:, :, :self.d_model-1]
        
        return torch.cat((linear_part, periodic_part), -1)  # Concatenate linear and periodic parts

In [ ]:
# class TimeSeriesTransformer(nn.Module):
#     def __init__(self, input_dim, d_model, n_head, num_encoder_layers, dropout_prob, output_dim):
#         super(TimeSeriesTransformer, self).__init__()
#         self.time2vec = Time2Vec(d_model=time2vec_dim)  # Assuming time2vec_dim <= d_model
#         self.dropout = nn.Dropout(dropout_prob)
#         encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=dropout_prob, batch_first=True)
#         self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
#         self.output_layer = nn.Linear(d_model, output_dim)

#     def generate_square_subsequent_mask(self, sz):
#         mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
#         return mask

#     def forward(self, src):
#         time_indices = src.size(1)
#         # time_indices should be of shape [batch_size, seq_len, 1]
#         time_embeddings = self.time2vec(time_indices)  # Generate time embeddings
#         src = torch.cat((src, time_embeddings), -1)  # Concatenate src features with time embeddings
#         src = self.dropout(src)

#         mask = self.generate_square_subsequent_mask(src.size(1)).to(device)

#         output = self.transformer_encoder(src, mask=mask)  # Ensure src is [batch_size, seq_len, d_model] due to batch_first=True
#         output = self.dropout(output)
#         output = self.output_layer(output[:, -1, :])  # Taking the last time step; adjust as needed
#         return output

<img src="/home/arda/Turkcell/images/Screenshot from 2024-02-19 12-26-08.png" alt="Alt text">
<img src="/home/arda/Turkcell/images/Screenshot from 2024-02-19 12-26-54.png" alt="Alt text">
<img src="/home/arda/Turkcell/images/Screenshot from 2024-02-19 12-27-52.png" alt="Alt text">



In [ ]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model, n_head, num_encoder_layers, dropout_prob, output_dim):
        super(TimeSeriesTransformer, self).__init__()
        self.input_embedding = nn.Linear(input_dim, d_model)
        # self.dropout = nn.Dropout(dropout_prob)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=dropout_prob, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.output_layer = nn.Linear(d_model, output_dim)


    def forward(self, src):
        src = self.input_embedding(src).to(device)  # [batch_size, seq_len, input_dim] -> [batch_size, seq_len, d_model]
        # src = self.dropout(src)
        src = self.pos_encoder(src).to(device)
        # src = self.dropout(src)

        output = self.transformer_encoder(src)  # Ensure src is [batch_size, seq_len, d_model] due to batch_first=True
        # output = self.dropout(output)
        output = self.output_layer(output[:, -1, :])  # Taking the last time step; adjust as needed
        return output

In [ ]:
class ModelActioner:
    
    def __init__(self, train_data, test_data, device):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.model = None
        self.optimizer = None
        self.criterion = nn.MSELoss()

    
    def train_validate(self, config, trial):

        # batch_size = config["batch_size"]
        # epochs = config["epochs"]
        # d_model = config["d_model"]
        # num_encoder_layers = config["num_encoder_layers"]
        # n_head = config["n_head"]
        # learning_rate = config["learning_rate"]
        # dropout_prob = config["dropout_prob"]
        # weight_decay = config["weight_decay"]
        # lr_step_size = config["lr_step_size"]
        # gamma = config["gamma"]

        batch_size = config["batch_size"]
        epochs = config["epochs"]
        d_model = 512
        num_encoder_layers = config["num_encoder_layers"]
        n_head = 8
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = config["lr_step_size"]
        gamma = config["gamma"]


        n_splits = 3
        tscv = TimeSeriesSplit(n_splits=n_splits)

        val_losses = []

        for fold, (train_ids, val_ids) in enumerate(tscv.split(self.train_data)):
            print(f'Starting fold {fold+1}:')
            
            self.model = TimeSeriesTransformer(input_dim=self.train_data.__getitem__(0)[0].shape[1], d_model=d_model, n_head=n_head, num_encoder_layers=num_encoder_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

            self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate, weight_decay=weight_decay)

            scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True) 

            train_subset = Subset(self.train_data, train_ids)
            val_subset = Subset(self.train_data, val_ids)
            
            # Creating data loader
            train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=False)
            val_loader = DataLoader(dataset=val_subset, batch_size=batch_size, shuffle=False)

            # Training Loop
            for epoch in range(epochs):
                print('epochs {}/{}'.format(epoch+1,epochs))

                running_loss = 0.0
                total_sample_train = 0

                print("hidden none")

                # self.model.hidden_none()
                self.model.train()

                for batch_idx, (data, target) in enumerate(train_loader):
                    data, target = data.to(self.device), target.to(self.device)
                    target = target.view(-1,1) 

                    self.optimizer.zero_grad()
                    preds = self.model(data)

                    loss = self.criterion(preds, target)
                    loss.backward()
                    self.optimizer.step() # Update model params

                    running_loss += loss.item() * data.size(0)
                    total_sample_train += data.size(0)

                train_loss = running_loss/total_sample_train
                #print(f"train loss: {train_loss}")
                self.model.eval()
                val_running_loss = 0.0
                total_sample_val = 0

                with torch.no_grad():

                    for batch_idx, (data, target) in enumerate(val_loader):
                        data, target = data.to(self.device), target.to(self.device)
                        target = target.view(-1,1)

                        preds = self.model(data)
                        loss = self.criterion(preds, target)

                        val_running_loss += loss.item() * data.size(0)
                        total_sample_val += data.size(0)
                
                val_loss = val_running_loss/total_sample_val
                val_losses.append(val_loss)
                scheduler.step(val_loss)
                
                unique_step = fold * epochs + epoch
                trial.report(val_loss, unique_step)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                current_lr = self.optimizer.param_groups[0]['lr']

                print(f'Current Learning Rate: {current_lr}')
                print(f"train_loss: {train_loss}, val_loss: {val_loss}")
                
        mean_val_loss = np.mean(val_losses)
        print(f"Mean validation loss: {mean_val_loss}")
        return mean_val_loss
    
                    
    def train(self, config):

        # batch_size = config["batch_size"]
        # epochs = config["epochs"]
        # d_model = config["d_model"]
        # num_encoder_layers = config["num_encoder_layers"]
        # n_head = config["n_head"]
        # learning_rate = config["learning_rate"]
        # dropout_prob = config["dropout_prob"]
        # weight_decay = config["weight_decay"]
        # lr_step_size = config["lr_step_size"]
        # gamma = config["gamma"]

        batch_size = config["batch_size"]
        epochs = config["epochs"]
        d_model = 512
        num_encoder_layers = config["num_encoder_layers"]
        n_head = 8
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = config["lr_step_size"]
        gamma = config["gamma"]

        self.model = TimeSeriesTransformer(input_dim=self.train_data.__getitem__(0)[0].shape[1], d_model=d_model, n_head=n_head, num_encoder_layers=num_encoder_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

        # Update optimizer with updated lr
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate, weight_decay=weight_decay)

        # Creating data loader
        train_loader = DataLoader(dataset=self.train_data, batch_size=batch_size, shuffle=False)

        scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True)  

        # Training Loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))

            running_loss = 0.0
            total_sample_train = 0

            self.model.train()

            for batch_idx, (data, target) in enumerate(train_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)  

                self.optimizer.zero_grad()
                preds = self.model(data)

                loss = self.criterion(preds, target)
                #loss = loss.mean()
                loss.backward()
                self.optimizer.step() # Update model params

                running_loss += loss.item() * data.size(0)
                total_sample_train += data.size(0)

            train_loss = running_loss/total_sample_train
            #print(f"train loss: {train_loss}")
            scheduler.step(train_loss)
            current_lr = self.optimizer.param_groups[0]['lr']

            print(f'Current Learning Rate: {current_lr}')
            print(f"train_loss: {train_loss}")
        
        return self.model
            
    
    def test(self, config):
        batch_size = config["batch_size"]
        all_preds = []

        test_loader = DataLoader(dataset=self.test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0
        total_sample = 0

        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(test_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)
                
                preds = self.model(data)
                loss = self.criterion(preds, target)

                running_loss += loss.item() * data.size(0)
                total_sample += data.size(0)

                all_preds.extend(preds.cpu().numpy())

            test_loss = running_loss/total_sample
            print(f"test_loss: {test_loss}")

        return all_preds
    


In [ ]:

def objective(trial):
    config = {
        "batch_size": trial.suggest_int("batch_size", 32, 128, log=True),
        "epochs": trial.suggest_int("epochs", 10, 100),
        # "d_model": trial.suggest_int("d_model", 10, 150),
        "num_encoder_layers": trial.suggest_int("num_encoder_layers", 1, 8),
        # "n_head": trial.suggest_int("n_head", 3, 6),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1),
        "dropout_prob": trial.suggest_float("dropout_prob", 0.0, 0.3),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
        "lr_step_size": trial.suggest_int("lr_step_size", 10, 100), 
        "gamma": trial.suggest_float("gamma", 0.1, 0.9)
    }

    trainer = ModelActioner(train_data, test_data, device)

    val_loss = trainer.train_validate(config, trial)

    return val_loss

In [ ]:
study_name = "Transformer-Tunner"
storage_url = "sqlite:///db.sqlite3"

storage = optuna.storages.RDBStorage(url=storage_url)

# Check if the study exists
study_names = [study.study_name for study in optuna.study.get_all_study_summaries(storage=storage)]
if study_name in study_names:
    # Delete the study if it exists
    print(f"Deleting study '{study_name}'")
    optuna.delete_study(study_name=study_name, storage=storage_url)
else:
    print(f"Study '{study_name}' does not exist in the storage.")
    

study = optuna.create_study(direction='minimize', 
                            storage=storage_url, 
                            sampler=TPESampler(),
                            pruner=optuna.pruners.HyperbandPruner(
                            min_resource=30,  
                            max_resource=100,  
                            reduction_factor=3,  
                            ),
                            study_name=study_name,
                            load_if_exists=False)

pbar = tqdm(total=15, desc='Optimizing', unit='trial')

def callback(study, trial):
    # Update the progress bar
    pbar.update(1)
    pbar.set_postfix_str(f"Best Value: {study.best_value:.4f}")


study.optimize(objective, n_trials=15, callbacks=[callback])
pbar.close()

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value:', trial.value)
print('Params:')
for key, value in trial.params.items():
    print(f'{key}: {value}')

In [ ]:
trainer = ModelActioner(train_data, test_data, device)
model = trainer.train(trial.params)

In [ ]:
preds = trainer.test(trial.params)
preds = np.array(preds)

y_true = y_test[time_step:]
y_true = scaler_adj.inverse_transform(y_true.reshape(-1, 1)).flatten()
preds = scaler_adj.inverse_transform(preds.reshape(-1, 1)).flatten()


mse = mean_squared_error(y_true, preds)
print(f'Mean Squared Error: {mse:.4f}')

rmse = mean_squared_error(y_true, preds, squared=False)
print(f'Root Mean Squared Error: {rmse:.4f}')

r2 = r2_score(y_true, preds)
print(f'R² Score: {r2:.4f}')

mape = mean_absolute_percentage_error(y_true, preds)*100
print(f'mape Score: % {mape:.4f}')

In [ ]:
len(preds)

In [ ]:
len(y_true)

In [ ]:
stock_price = stock_data["Adj Close"].iloc[test_index:]
stock_price=stock_price.reset_index()
stock_price=stock_price.drop(columns=["index"])
stock_price

In [ ]:
signals = pd.DataFrame(preds, columns=['pred'])
signals["next_day"] = pd.DataFrame(y_true)
signals["today"] = stock_price
signals["Signal_Pred"] = (signals["today"] < signals["pred"]).astype(int)
signals["Signal_True"] = (signals["today"] < signals["next_day"]).astype(int)

signals

In [ ]:
signals["Signal_Pred"].value_counts()

In [ ]:
signals["Date"] = date_test
signals

In [ ]:
stock_price = stock_data["Adj Close"].iloc[test_index:]
stock_price=stock_price.reset_index()
stock_price=stock_price.drop(columns=["index"])
stock_price

In [ ]:
date_test["Date"] = date_test["Date"].dt.strftime('%Y-%m-%d')
date_test

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(date_test).flatten(), y=stock_data["Adj Close"].iloc[test_index:], mode='lines', name='TSLA Stock Price'))

# Buy and sell signals
buy_signals = signals[signals['Signal_Pred'] == 1]
sell_signals = signals[signals['Signal_Pred'] == 0]

# Ensure that buy and sell prices are aligned with the signals by matching on the 'Date' column
buy_prices = stock_data[stock_data['Date'].isin(buy_signals['Date'])]["Adj Close"]
sell_prices = stock_data[stock_data['Date'].isin(sell_signals['Date'])]["Adj Close"]


# Plot buy signals
fig.add_trace(go.Scatter(
    x=buy_signals['Date'], 
    y=buy_prices, 
    mode='markers', 
    name='Buy', 
    marker=dict(color='green', size=10, symbol='triangle-up')
))


# Plot sell signals
fig.add_trace(go.Scatter(
    x=sell_signals['Date'], 
    y=sell_prices, 
    mode='markers', 
    name='Sell', 
    marker=dict(color='red', size=10, symbol='triangle-down')
))


# Update layout
fig.update_layout(
    title='Stock Price with Buy and Sell Signals',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    height = 700,
    width=1280
)

# Show the plot
pyo.iplot(fig)

In [ ]:
trace_pred = go.Scatter(
    x=np.array(date_test).flatten(),
    y=signals['pred'],
    mode='lines+markers',
    name='Predicted'
)

trace_true = go.Scatter(
    x=np.array(date_test).flatten(),
    y=signals['next_day'],
    mode='lines+markers',
    name='Actual Next Day'
)

# Define the layout
layout = go.Layout(
    title='Predicted vs Actual Next Day Values',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Value'),
    height=700
)

# Create the figure and add traces
fig = go.Figure(data=[trace_pred, trace_true], layout=layout)

# Show the figure
fig.show()